In [8]:
%pip install selenium webdriver-manager beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time


In [10]:
def scrape_tokopedia(query):
    options = Options()
    # options.add_argument("--headless")  # jalanin tanpa buka window
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=Service(), options=options)
    url = f"https://www.tokopedia.com/search?st=product&q={query.replace(' ', '%20')}"
    driver.get(url)

    time.sleep(5) 

    results = []

    cards = driver.find_elements(By.CSS_SELECTOR, 'a[data-theme="default"]')[:4]  
    for card in cards:
        try:
            link = card.get_attribute('href')
            
            # Nama toko
            try:
                toko = card.find_element(By.CSS_SELECTOR, 'span[class*="T0rpy-LEw"]').text
            except:
                toko = "Unknown"
                if link:
                    temp = link.replace("https://www.tokopedia.com/", "")
                    toko = temp.split("/", 1)[0]
                    toko = toko.replace("-", " ")
            
            # Nama produk
            try:
                nama_produk = card.find_element(By.CSS_SELECTOR, 'span[class*="0T8-iGxMp"]').text
            except:
                nama_produk = query
                if link:
                    temp = link.replace(f"https://www.tokopedia.com/{toko}/", "")
                    nama_produk = temp.replace("-", " ")

            # Gambar
            try:
                image = card.find_element(By.TAG_NAME, 'img').get_attribute('src')
            except:
                image = "Image"

            # Harga 
            try:
                harga = card.find_element(By.CSS_SELECTOR, 'div[class*="67d6E1xDK"]').text
            except:
                harga = "Rp. -"

            results.append({
                'Nama Produk': nama_produk,
                'Harga': harga,
                'Toko': toko,
                'Gambar': image,
                'Link': link
            })
        except Exception as e:
            print("Error parsing card:", e)
            continue

    driver.quit()
    return results


In [13]:
input = "Iphone 13"
hasil = scrape_tokopedia(input)
for item in hasil:
    print(item)

{'Nama Produk': 'Apple iPhone 13 Garansi Resmi - 128GB 256GB 512GB - 128GB, Midnight Black', 'Harga': 'Rp8.349.000', 'Toko': 'ismile indonesia', 'Gambar': 'https://images.tokopedia.net/img/cache/200-square/VqbcmM/2024/8/15/e426b597-a171-4d23-9a93-87913ec825bf.png.webp?ect=4g', 'Link': 'https://www.tokopedia.com/ismile-indonesia/apple-iphone-13-garansi-resmi-128gb-256gb-512gb-128gb-midnight-black-8c030'}
{'Nama Produk': 'Apple iPhone 13 128GB 256GB 512GB Garansi Resmi - 128GB, Midnight Black', 'Harga': 'Rp8.349.000', 'Toko': 'ismile indonesia', 'Gambar': 'https://images.tokopedia.net/img/cache/200-square/VqbcmM/2025/3/6/fb66a321-837f-4571-9c60-114d2e46ce32.jpg.webp?ect=4g', 'Link': 'https://www.tokopedia.com/ismile-indonesia/apple-iphone-13-128gb-256gb-512gb-garansi-resmi-128gb-midnight-black-6c20c'}
{'Nama Produk': 'APPLE iPhone 13 128GB 256GB 512GB A15 Bionic CPU 6C / GPU 4C Resmi IBOX', 'Harga': 'Rp8.129.000', 'Toko': 'Collins Official', 'Gambar': 'https://images.tokopedia.net/img/ca

In [16]:
import pandas as pd
import re

def save_to_excel(data, query):
    clean_query = re.sub(r'[\\/*?:"<>|]', "", query).strip().replace(" ", "-")
    filename = f"scraping-product-{clean_query}.xlsx"

    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    print(f"Data berhasil disimpan ke {filename}")

save_to_excel(hasil, input)

Data berhasil disimpan ke scraping-product-Iphone-13.xlsx
